In [64]:
import pandas as pd
import seaborn as sns
from urllib.request import urlretrieve
import re

In [65]:
table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

# Read the table using Pandas
metadata_df = pd.read_parquet('metadata.parquet')

In [66]:
metadata_df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547


In [138]:

# Our harm dictionary
# Organized as so: [HarmScore, Violent (0-1), Explicit (0-1)] 
harmDict = {
    # High Harm - NSFW / Explicit
    "porn": [10, 0, 1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [6, 0, 1],
    "creampie": [7, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "hentai": [9, 0, 1],
    "lolicon": [10, 0, 1],
    "moaning": [4, 0, 1],
    "nsfw": [6, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "sexually": [8, 0, 1],
    "intercourse": [9, 0, 1],
    "making love": [6, 0, 1],
    "have sex": [9, 0, 1],
    "having sex": [9, 0, 1],
    "sex scene": [8, 0, 1],
    "sexual act": [8, 0, 1],
    "sexual contact": [7, 0, 1],
    "sexual activity": [7, 0, 1],
    "sexually explicit": [10, 0, 1],
    "sexual pleasure": [8, 0, 1],
    "sexualized": [6, 0, 1],
    "sexualizing": [6, 0, 1],
    "sex position": [7, 0, 1],
    "sex doll": [8, 0, 1],
    "sex appeal": [5, 0, 1],
    "sex drive": [5, 0, 1],

    # Euphemisms / Slang
    "bang": [7, 0, 1],
    "banging": [7, 0, 1],
    "smash": [6, 0, 1],
    "smashing": [6, 0, 1],
    "hookup": [6, 0, 1],
    "get laid": [8, 0, 1],
    "doing it": [6, 0, 1],
    "boning": [6, 0, 1],
    "fucking": [9, 0, 1],
    "fuck": [9, 0, 1],
    "screwing": [6, 0, 1],
    "shagging": [6, 0, 1],
    "humping": [6, 0, 1],
    "getting busy": [6, 0, 1],
    "ride him": [7, 0, 1],
    "ride her": [7, 0, 1],
    "tapped that": [7, 0, 1],
    "clapped": [7, 0, 1],
    "piping": [6, 0, 1],

    # Descriptive & Implied NSFW
    "intimate encounter": [6, 0, 1],
    "romantic entanglement": [5, 0, 1],
    "carnal delight": [7, 0, 1],
    "heated moment": [5, 0, 1],
    "erotic act": [8, 0, 1],
    "lovers embrace": [5, 0, 1],
    "bodies collide": [7, 0, 1],
    "flesh against flesh": [8, 0, 1],
    "night of passion": [6, 0, 1],
    "under the covers": [5, 0, 1],
    "between the sheets": [6, 0, 1],

    # Anatomical / Fetish
    "tits": [6, 0, 1],
    "milf": [7, 0, 1],
    "anal": [7, 0, 1],
    "asshole": [6, 0, 1],
    "cock": [6, 0, 1],
    "cum": [6, 0, 1],
    "cumming": [6, 0, 1],
    "cunnilingus": [7, 0, 1],
    "dildo": [6, 0, 1],
    "ejaculation": [6, 0, 1],
    "fingering": [6, 0, 1],
    "fisting": [6, 0, 1],
    "gag": [5, 0, 1],
    "gangbang": [6, 0, 1],
    "handjob": [6, 0, 1],
    "kissing": [6, 0, 1],
    "lube": [6, 0, 1],
    "masturbation": [6, 0, 1],
    "nipple": [6, 0, 1],
    "nipples": [6, 0, 1],
    "cleavage": [6, 0, 1],
    "fellatio": [7, 0, 1],
    "footjob": [6, 0, 1],
    "paizuri": [8, 0, 1],
    "tentacle": [6, 0, 1],
    "vagina": [7, 0, 1],
    "penis": [7, 0, 1],
    "dick": [7, 0, 1],
    "butt": [5, 0, 1],
    "underwear": [3, 0, 1],
    "lingerie": [5, 0, 1],

    # Violence & Extreme
    "rape": [10, 1, 1],
    "incest": [10, 1, 1],
    "pedophilia": [10, 1, 1],
    "necrophilia": [10, 1, 1],
    "zoophilia": [10, 1, 1],
    "scat": [9, 1, 1],
    "snuff": [10, 1, 1],
    "bukkake": [8, 1, 1],
    "gore": [8, 1, 0],

    # Borderline / Meta / Contextual
    "fursona": [7, 0, 1],
    "lolita": [10, 0, 0],
    "yaoi": [10, 0, 0],
    "yuri": [10, 0, 0],
    "loli": [10, 0, 0],
    "attractive": [2, 0, 0],
    "sexy": [3, 0, 1],
    "sensual": [4, 0, 0],
    "submissive": [4, 0, 0],
    "swimsuit": [2, 0, 0],
    "swimsuits": [2, 0, 0],
    "swimwear": [2, 0, 0],

    # Additional Descriptors
    "nudity": [8, 0, 1],
    "naked": [8, 0, 1],
    "topless": [7, 0, 1],
    "bare chest": [6, 0, 1],
    "bare breasts": [7, 0, 1],
    "bare skin": [5, 0, 1],
    "exposed breasts": [7, 0, 1],
    "exposed chest": [6, 0, 1],
    "bare butt": [6, 0, 1],
    "barely clothed": [5, 0, 1],
    "undressed": [6, 0, 1],
    "stripped": [6, 0, 1],
    "stripping": [6, 0, 1],
    "disrobed": [6, 0, 1],
    "no clothing": [6, 0, 1],
    "without clothes": [6, 0, 1],
    "exposed nudity": [9, 0, 1],
    "full frontal": [9, 0, 1],
    "full nude": [9, 0, 1],
    "completely naked": [9, 0, 1],
    "scantily clad": [6, 0, 1],
    "partially nude": [7, 0, 1],
    "public nudity": [8, 0, 1]
}

innoTerms = {
    # Innocent / Youth Descriptors
    "child": [1, 0, 0],
    "children": [1, 0, 0],
    "kid": [1, 0, 0],
    "kids": [1, 0, 0],
    "toddler": [1, 0, 0],
    "infant": [1, 0, 0],
    "newborn": [1, 0, 0],
    "baby": [1, 0, 0],
    "minor": [1, 0, 0],
    "preteen": [1, 0, 0],
    "teen": [1, 0, 0],
    "teenager": [1, 0, 0],
    "youngster": [1, 0, 0],
    "adolescent": [1, 0, 0],
    "juvenile": [1, 0, 0],
    "grade schooler": [1, 0, 0],
    "girl": [1, 0, 0],
    "girls": [1, 0, 0],
    "boy": [1, 0, 0],
    "boys": [1, 0, 0],
    "young girl": [1, 0, 0],
    "young boy": [1, 0, 0],
    "little girl": [1, 0, 0],
    "little boy": [1, 0, 0],
    "teen girl": [1, 0, 0],
    "teen boy": [1, 0, 0],
    "schoolgirl": [1, 0, 0],
    "schoolgirls": [1, 0, 0],
    "schoolboy": [1, 0, 0],
    "schoolboys": [1, 0, 0],
    "daughter": [1, 0, 0],
    "son": [1, 0, 0],
    "niece": [1, 0, 0],
    "nephew": [1, 0, 0],
    "innocent": [1, 0, 0],
    "naive": [1, 0, 0],
    "virgin": [2, 0, 0],
    "pure": [1, 0, 0],
    "unaware": [1, 0, 0],
    "curious": [1, 0, 0],
    "chaste": [1, 0, 0],
    "young body": [2, 0, 0],
    "emma watson": [4, 0, 0],
    "woman": [1, 0, 0],
    "women": [1, 0, 0],
    "man": [1, 0, 0],
    "men": [1, 0, 0],
    "adult": [1, 0, 0],
    "adults": [1, 0, 0],
    "lady": [1, 0, 0],
    "ladies": [1, 0, 0],
    "gentleman": [1, 0, 0],
    "gentlemen": [1, 0, 0],
    "female": [1, 0, 0],
    "females": [1, 0, 0],
    "male": [1, 0, 0],
    "males": [1, 0, 0],
    "mother": [1, 0, 0],
    "father": [1, 0, 0],
    "mom": [1, 0, 0],
    "dad": [1, 0, 0],
    "bride": [1, 0, 0],
    "groom": [1, 0, 0],
    "wife": [1, 0, 0],
    "husband": [1, 0, 0]    
}

adjTerms = {
    # Pose / Action Descriptors
    "kneeling": [3, 0, 1],
    "spreading": [4, 0, 1],
    "arched back": [4, 0, 1],
    "bending over": [4, 0, 1],
    "squatting": [3, 0, 1],
    "crawling": [2, 0, 1],
    "laying down": [2, 0, 1],
    "posing": [2, 0, 1],
    "leaning forward": [3, 0, 1],
    "hands behind back": [3, 0, 1],
    "on all fours": [5, 0, 1],
    "legs open": [5, 0, 1],
    "legs spread": [5, 0, 1],

    # Mood / Tone
    "submissive": [4, 0, 1],
    "dominant": [4, 0, 1],
    "obedient": [3, 0, 1],
    "shy": [3, 0, 1],
    "blushing": [3, 0, 1],
    "eager": [2, 0, 1],
    "seductive": [4, 0, 1],
    "flustered": [3, 0, 1],
    "nervous": [2, 0, 1],
    "tempting": [3, 0, 1],
    "teasing": [3, 0, 1],

    # Physical / Body Visuals
    "tight": [4, 0, 1],
    "wet": [4, 0, 1],
    "moist": [4, 0, 1],
    "bare": [3, 0, 1],
    "smooth": [2, 0, 1],
    "delicate": [2, 0, 1],
    "undeveloped": [3, 0, 1],
    "flat": [2, 0, 1],
    "tiny": [2, 0, 1],
    "fragile": [2, 0, 1],
    "slim": [2, 0, 1],
    "soft": [2, 0, 1],
    "exposed": [4, 0, 1],
    "revealing": [3, 0, 1],
    "skimpy": [4, 0, 1],
    "see-through": [4, 0, 1],
    "transparent": [4, 0, 1],
    "skin-tight": [4, 0, 1],
    "suggestive": [4, 0, 1],
    "seductively": [4, 0, 1],
    "suggestively": [4, 0, 1],
    "provocatively": [4, 0, 1],
    "flirtatiously": [3, 0, 1],
    "sensually": [4, 0, 1],
    "teasingly": [3, 0, 1],
    "shyly": [3, 0, 1],
    "lustfully": [4, 0, 1],
    "invitingly": [3, 0, 1],
    "sexually": [5, 0, 1],  # borderline strong, may also exist in harmDict
    "arousingly": [4, 0, 1],
    "passionately": [3, 0, 1],
    "naughtily": [4, 0, 1],
    "temptingly": [3, 0, 1]
}


def contains_phrases(prompt, dictionary):
    prompt_clean = re.sub(r"[^\w\s]", "", prompt.lower())  # normalize punctuation
    found = set()
    for phrase in dictionary:
        if " " in phrase and phrase in prompt_clean:
            found.add(phrase)
    return found


def getPromptHarmScore(prompt, harm_map, inno_map, adj_map):
    import re
    prompt = prompt.lower()

    # Use regex to strip punctuation and tokenize
    tokens = re.findall(r"\b\w[\w\-']*\b", prompt)
    unique_tokens = set(tokens)

    # Single-word matches
    harm_tokens = {t for t in unique_tokens if t in harm_map}
    inno_tokens = {t for t in unique_tokens if t in inno_map}
    adj_tokens = {t for t in unique_tokens if t in adj_map}

    # Clean prompt for phrase matching
    prompt_clean = re.sub(r"[^\w\s]", "", prompt)

    # Phrase matches (multi-word)
    harm_phrases = contains_phrases(prompt_clean, harm_map)
    inno_phrases = contains_phrases(prompt_clean, inno_map)
    adj_phrases = contains_phrases(prompt_clean, adj_map)

    # Combine hits
    harm_found = harm_tokens.union(harm_phrases)
    inno_found = inno_tokens.union(inno_phrases)
    adj_found = adj_tokens.union(adj_phrases)

    total_score = 0

    # Rule 1
    if harm_found:
        for t in harm_found:
            total_score += harm_map[t][0]
        for t in inno_found:
            total_score += inno_map[t][0]
        for t in adj_found:
            total_score += adj_map[t][0]

    # Rule 2
    elif inno_found and adj_found:
        for t in inno_found:
            total_score += inno_map[t][0]
        for t in adj_found:
            total_score += adj_map[t][0]

    # Rule 3
    else:
        total_score = 0

    return total_score, list(harm_found), list(inno_found), list(adj_found)



#Calculating the harm score for each prompt
metadata_df["harm_score"] = metadata_df["prompt"].apply(
    lambda x: getPromptHarmScore(x, harmDict, innoTerms, adjTerms)
)

print("Finished")


Finished


In [139]:
metadata_df['harm_score'].max()

(26,
 ['underwear', 'submissive', 'bondage'],
 ['emma watson'],
 ['submissive', 'kneeling'])

In [140]:
filteredData = metadata_df.drop_duplicates(subset=["prompt"], keep="first", inplace=True)

In [141]:
AccendfilteredData = metadata_df.sort_values(by="harm_score", ascending=False)

In [142]:
AccendfilteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
690634,0bc4de3b-bbd1-4f6d-a26d-91055b2699d3.png,full shot portrait of very very beautiful emma...,691,3596561265,150,14.0,8,512,640,dfea04daf2d7ef2d43228fd60abbc7d7c96dce721a6ef6...,2022-08-14 01:29:00+00:00,0.927999,0.003402,"(26, [underwear, submissive, bondage], [emma w..."
121304,3ec4f3cb-37d0-4cd8-8558-1bb302f2997a.png,Sensual beautiful perfect Emma Watson as Megum...,122,3161342213,120,12.0,8,512,640,525601af5de9c6c772158b6e9d1103a295761d5e34b8f0...,2022-08-10 18:38:00+00:00,0.074697,0.001580,"(26, [sensual, loli], [emma watson], [slim, se..."
121303,b5122d10-f022-4a74-9495-a1bf186a2235.png,Sensual beautiful perfect Emma Watson as Megum...,122,1114709948,120,12.0,8,512,640,525601af5de9c6c772158b6e9d1103a295761d5e34b8f0...,2022-08-10 18:37:00+00:00,0.112385,0.001599,"(26, [sensual, loli], [emma watson], [slim, se..."
121302,e5151926-accb-4256-a5ec-c18796426d6f.png,Sensual beautiful perfect Emma Watson as Megum...,122,861749965,120,12.0,8,512,640,525601af5de9c6c772158b6e9d1103a295761d5e34b8f0...,2022-08-10 18:36:00+00:00,0.090628,0.001566,"(26, [sensual, loli], [emma watson], [slim, se..."
1804417,67e9ef57-6439-4c2f-a30b-6924c7af6da1.png,"creampie, bukkake, gokkun, milf, gonzo, bare, ...",1805,2567510302,150,15.0,8,512,768,7504a6c17d2e9d29d86b361909dc409b19ae8f13958369...,2022-08-11 08:57:00+00:00,0.081520,0.924438,"(25, [milf, bukkake, creampie], [], [bare])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750888,70714254-258a-447c-b8a9-4e2e78092c04.png,a beautiful and androgynous half - elf with da...,751,2977493072,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:18:00+00:00,0.332212,0.002896,"(0, [], [], [])"
750887,a7ac616f-b55a-443a-8462-fb5475a035f6.png,a beautiful and androgynous half - elf with da...,751,1473583392,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:04:00+00:00,0.391229,0.002861,"(0, [], [], [])"
750886,dbae0baf-b32e-41d7-b247-979cad45a066.png,a beautiful and androgynous half - elf with al...,751,1378649036,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:02:00+00:00,0.418952,0.001591,"(0, [], [], [])"
750885,1bfce50b-57c6-4a25-b5ae-ff0c4778f3ee.png,a beautiful and androgynous half - elf with al...,751,3173024222,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:01:00+00:00,0.739727,0.001493,"(0, [], [], [])"


In [143]:
AccendfilteredData["HarmLabel"] = AccendfilteredData["harm_score"].apply(
    lambda x: "Harmful" if x[0] >= 5 else "Harmless"
)


In [144]:
AccendfilteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score,HarmLabel
690634,0bc4de3b-bbd1-4f6d-a26d-91055b2699d3.png,full shot portrait of very very beautiful emma...,691,3596561265,150,14.0,8,512,640,dfea04daf2d7ef2d43228fd60abbc7d7c96dce721a6ef6...,2022-08-14 01:29:00+00:00,0.927999,0.003402,"(26, [underwear, submissive, bondage], [emma w...",Harmful
121304,3ec4f3cb-37d0-4cd8-8558-1bb302f2997a.png,Sensual beautiful perfect Emma Watson as Megum...,122,3161342213,120,12.0,8,512,640,525601af5de9c6c772158b6e9d1103a295761d5e34b8f0...,2022-08-10 18:38:00+00:00,0.074697,0.001580,"(26, [sensual, loli], [emma watson], [slim, se...",Harmful
121303,b5122d10-f022-4a74-9495-a1bf186a2235.png,Sensual beautiful perfect Emma Watson as Megum...,122,1114709948,120,12.0,8,512,640,525601af5de9c6c772158b6e9d1103a295761d5e34b8f0...,2022-08-10 18:37:00+00:00,0.112385,0.001599,"(26, [sensual, loli], [emma watson], [slim, se...",Harmful
121302,e5151926-accb-4256-a5ec-c18796426d6f.png,Sensual beautiful perfect Emma Watson as Megum...,122,861749965,120,12.0,8,512,640,525601af5de9c6c772158b6e9d1103a295761d5e34b8f0...,2022-08-10 18:36:00+00:00,0.090628,0.001566,"(26, [sensual, loli], [emma watson], [slim, se...",Harmful
1804417,67e9ef57-6439-4c2f-a30b-6924c7af6da1.png,"creampie, bukkake, gokkun, milf, gonzo, bare, ...",1805,2567510302,150,15.0,8,512,768,7504a6c17d2e9d29d86b361909dc409b19ae8f13958369...,2022-08-11 08:57:00+00:00,0.081520,0.924438,"(25, [milf, bukkake, creampie], [], [bare])",Harmful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750888,70714254-258a-447c-b8a9-4e2e78092c04.png,a beautiful and androgynous half - elf with da...,751,2977493072,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:18:00+00:00,0.332212,0.002896,"(0, [], [], [])",Harmless
750887,a7ac616f-b55a-443a-8462-fb5475a035f6.png,a beautiful and androgynous half - elf with da...,751,1473583392,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:04:00+00:00,0.391229,0.002861,"(0, [], [], [])",Harmless
750886,dbae0baf-b32e-41d7-b247-979cad45a066.png,a beautiful and androgynous half - elf with al...,751,1378649036,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:02:00+00:00,0.418952,0.001591,"(0, [], [], [])",Harmless
750885,1bfce50b-57c6-4a25-b5ae-ff0c4778f3ee.png,a beautiful and androgynous half - elf with al...,751,3173024222,50,7.0,8,512,512,ea7e74d37a9cbe6241d4ede73391bce024d4fca992e9a0...,2022-08-12 07:01:00+00:00,0.739727,0.001493,"(0, [], [], [])",Harmless


In [149]:
test = input("Enter your Prompt!")
getPromptHarmScore(test, harmDict, innoTerms, adjTerms)




(14, ['ejaculation', 'penis'], ['woman'], [])